In [1]:
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [2]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader,Dataset
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import FusionEmedding,GlyphEmbedding,PinyinEmbedding
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
glyph = GlyphEmbedding('data/glyph_weight.npy')
pinyinV = PinyinEmbedding(30,'data/pinyin_map.json')
# Example
display(glyph(torch.tensor([0,1,2])))
display(pinyinV(torch.tensor([0,1])))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0421, -0.0327,  0.0096,  ...,  0.0711,  0.0417, -0.0224],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1162, -0.0063, -0.1702, -0.0711,  0.0213,  0.0390, -0.2001, -0.1429,
          0.2429,  0.0553, -0.2503, -0.0788, -0.2204, -0.2940,  0.0712, -0.2851,
          0.1147, -0.2163,  0.2020,  0.2989,  0.3067,  0.2864, -0.0451,  0.0571,
          0.0190,  0.0826, -0.1811,  0.3057, -0.2350, -0.2527]],
       grad_fn=<EmbeddingBackward>)

In [6]:
# Example
_ = tokenizer('[UNK]辛[UNK]业怠π',return_tensors='pt')
display(_)
Anchibert(**_).last_hidden_state.shape

{'input_ids': tensor([[ 101,  100, 6789,  100,  689, 2591,  223,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

torch.Size([1, 8, 768])

In [7]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

In [11]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [12]:
trainSet = FusionDataset(tr_in[:110],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[:110],tr_pos_in[:110],tr_pos_out[:110])

110it [00:00, 2065.60it/s]
110it [00:00, 2233.97it/s]


In [13]:
trainLoader = DataLoader(trainSet,batch_size=64,shuffle=True)
#     validLoader = DataLoader(validSet,batch_size=batch_size,shuffle=False)

In [17]:
Anchibert.to(device)
for Xsents_input_ids,Xsents_token_type_ids,\
    Xsents_attention_mask,Xsents_pinyin_ids,\
    Xsents_glyph_ids,Xsents_pos_ids,\
    Ysents_input_ids,Ysents_token_type_ids,\
    Ysents_attention_mask,Ysents_pinyin_ids,\
    Ysents_glyph_ids,Ysents_pos_ids in trainLoader:
    
    Xsents_input_ids = Xsents_input_ids.to(device)
    Xsents_token_type_ids = Xsents_token_type_ids.to(device)
    Xsents_attention_mask = Xsents_attention_mask.to(device)
    Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
    Xsents_glyph_ids = Xsents_glyph_ids.to(device)
    Xsents_pos_ids = Xsents_pos_ids.to(device)
    Ysents_input_ids = Ysents_input_ids.to(device)
    Ysents_token_type_ids = Ysents_token_type_ids.to(device)
    Ysents_attention_mask = Ysents_attention_mask.to(device)
    Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
    Ysents_glyph_ids = Ysents_glyph_ids.to(device)
    Ysents_pos_ids = Ysents_pos_ids.to(device)
    
    Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                )['last_hidden_state'].detach()
    outputembeddingsX = model(Xword_embeddings,Xsents_pinyin_ids, \
                             Xsents_token_type_ids,Xsents_pos_ids)

    Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                 Ysents_token_type_ids, \
                                 Ysents_attention_mask  \
                                )['last_hidden_state'].detach()
    
    outputembeddingsY = model(Yword_embeddings,Ysents_pinyin_ids, \
                             Ysents_token_type_ids,Ysents_pos_ids)
    break

In [18]:
outputembeddingsX.shape,outputembeddingsY.shape

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(torch.Size([64, 29, 768]), torch.Size([64, 28, 768]))